# Triangulation

In [1]:
import os
import pickle
import numpy as np
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from time import time
from lib import misc, utils, app
from lib.calib import triangulate_points_fisheye

plt.style.use(os.path.join('..', 'configs', 'mplstyle.yaml'))

%load_ext autoreload
%autoreload 2

ROOT_DATA_DIR = os.path.join("/Users/matthewterblanche/Downloads/Baboon Data/AcinoSet-main","data")

# Reconstruction Params
Define the params in the cell below. Thereafter, run all cells

In [2]:
DATA_DIR = os.path.join(ROOT_DATA_DIR, "2021_10_07", "Baboon1", "walk")

start_frame = 24320
end_frame = 24705

# DLC p_cutoff - any points with likelihood < dlc_thresh are not trusted in optimisation
dlc_thresh = 0.5 # change this only if the optimisation result is unsatisfactory

# Reconstruction

In [3]:
assert os.path.exists(DATA_DIR)
OUT_DIR = os.path.join(DATA_DIR, 'tri')
DLC_DIR = os.path.join(DATA_DIR, 'dlc')
assert os.path.exists(DLC_DIR)
os.makedirs(OUT_DIR, exist_ok=True)

# load video info
res, fps, tot_frames, _ = app.get_vid_info(DATA_DIR) # path to original videos
assert end_frame <= tot_frames, f'end_frame must be less than or equal to {tot_frames}'

start_frame -= 1 # 0 based indexing
assert start_frame >= 0
N = end_frame-start_frame


def rot_y(y):
    c = sp.cos(y)
    s = sp.sin(y)
    return sp.Matrix([
        [c, 0, -s],
        [0, 1, 0],
        [s, 0, c]
    ])

#Load rotations csv

rotation_csv_path = os.path.join(DATA_DIR, 'encoder6_rad.csv')

rot_angles = []


rotation = utils.load_rotation(rotation_csv_path)

for i in rotation['Rotation (rad)'].values:
    rot_angles.append(np.float32(rot_y(i)))

k_arr, d_arr, r_arr, t_arr, cam_res, n_cams, scene_fpath = utils.find_scene_file(DATA_DIR, verbose=False)


print(r_arr)
dlc_points_fpaths = sorted(glob(os.path.join(DLC_DIR, '*.h5')))
assert n_cams == len(dlc_points_fpaths)
    
# Load Measurement Data (pixels, likelihood)
points_2d_df = utils.load_dlc_points_as_df(dlc_points_fpaths, verbose=False)
points_2d_df = points_2d_df[points_2d_df["frame"].between(start_frame, end_frame-1)]
points_2d_df = points_2d_df[points_2d_df['likelihood']>dlc_thresh] # ignore points with low likelihood

assert len(k_arr) == points_2d_df['camera'].nunique()


dfs = []
for i in range(start_frame,end_frame-1,1):
    print(i)
 #   rot_Arr = np.stack((rot_angles[i].T*r_arr[0],r_arr[1]))
   # print("R: ",rot_Arr)
    points_3d_df = (utils.get_pairwise_3d_points_from_df(
            points_2d_df[points_2d_df['frame']==i],
            k_arr, d_arr.reshape((-1,4)), r_arr, t_arr,
            triangulate_points_fisheye))    #print(dlc_df.columns)
    points_3d_df.columns.name = ''
    dfs.append(points_3d_df)
#create new dataframe
points_3d_df = pd.DataFrame(columns=['frame', 'marker', 'x', 'y', 'z'])
for i, df in enumerate(dfs):
    df['camera'] = i
    points_3d_df = pd.concat([points_3d_df, df], sort=True, ignore_index=True)

points_3d_df = points_3d_df[['frame', 'marker', 'x', 'y', 'z']]
points_3d_df['point_index'] = points_3d_df.index

#print(f'DLC points dataframe:\n{points_3d_df}\n')

[[[ 9.98628943e-01  4.01791010e-02 -3.35540614e-02]
  [-3.35540190e-02 -6.75602841e-04 -9.99436677e-01]
  [-4.01791364e-02  9.99192265e-01  6.73493767e-04]]

 [[ 9.99061483e-01  4.31081273e-02  4.22394622e-03]
  [ 4.58950813e-03 -8.38467301e-03 -9.99954316e-01]
  [-4.30707416e-02  9.99035228e-01 -8.57464896e-03]]]
24319

24320

24321

24322

24323

24324

24325

24326

24327

24328

24329

24330

24331

24332

24333

24334

24335

24336

24337

24338

24339

24340

24341

24342

24343

24344

24345

24346

24347

24348

24349

24350

24351

24352

24353

24354

24355

24356

24357

24358

24359

24360

24361

24362

24363

24364

24365

24366

24367

24368

24369

24370

24371

24372

24373

24374

24375

24376

24377

24378

24379

24380

24381

24382

24383

24384

24385

24386

24387

24388

24389

24390

24391

24392

24393

24394

24395

24396

24397

24398

24399

24400

24401

24402

24403

24404

24405

24406

24407

24408

24409

24410

24411

24412

24413

24414

24415

24416

# Save triangulation results

In [4]:
markers = misc.get_markers()

positions = np.full((N, len(markers), 3), np.nan)
for i, marker in enumerate(markers):
    marker_pts = points_3d_df[points_3d_df["marker"]==marker][["frame", "x", "y", "z"]].values
    for frame, *pt_3d in marker_pts:
        positions[int(frame)-start_frame, i] = pt_3d

app.save_tri(positions, OUT_DIR, scene_fpath, start_frame, dlc_thresh)

Saved /Users/matthewterblanche/Downloads/Baboon Data/AcinoSet-main/data/2021_10_07/Baboon1/walk/tri/tri.pickle
Saved /Users/matthewterblanche/Downloads/Baboon Data/AcinoSet-main/data/2021_10_07/Baboon1/walk/tri/tri.mat
Saved /Users/matthewterblanche/Downloads/Baboon Data/AcinoSet-main/data/2021_10_07/Baboon1/walk/tri/cam*_tri.h5
Saved /Users/matthewterblanche/Downloads/Baboon Data/AcinoSet-main/data/2021_10_07/Baboon1/walk/tri/cam*_tri.csv

Saving labeled videos...


  0%|          | 2/25627 [00:00<22:17, 19.16it/s]

Loading cam2 and data.

Duration of video: 427.12 s, recorded with 60.0 fps!
Total frames: 25627 with frame dimensions: 1920 x 1080
Generating frames and creating video...
Loading cam1 and data.

Duration of video: 427.12 s, recorded with 60.0 fps!
Total frames: 25627 with frame dimensions: 1920 x 1080
Generating frames and creating video...


100%|█████████▉| 25615/25627 [08:05<00:00, 61.51it/s]

Done!



100%|██████████| 25627/25627 [08:05<00:00, 52.78it/s]

# Plot the baboon!

In [5]:
data_fpath = os.path.join(OUT_DIR, 'tri.pickle')
#app.plot_baboon_reconstruction(data_fpath, dark_mode=True)
app.plot_baboon_reconstruction(data_fpath, reprojections=True, centered=False, dark_mode=True)

Loaded extrinsics from /Users/matthewterblanche/Downloads/Baboon Data/AcinoSet-main/data/2021_10_07/extrinsic_calib/2_cam_scene_sba.json



qt.qpa.window: <QNSWindow: 0x7fb99d22d530; contentView=<QNSView: 0x7fb99d22d090; QCocoaWindow(0x7fb99d22cf80, window=QWidgetWindow(0x7fb99d2142f0, name="QGLWidgetClassWindow"))>> has active key-value observers (KVO)! These will stop working now that the window is recreated, and will result in exceptions when the observers are removed. Break in QCocoaWindow::recreateWindowIfNeeded to debug.
/Users/matthewterblanche/opt/anaconda3/lib/python3.8/site-packages/pyqtgraph/graphicsItems/PlotCurveItem.py:153: RuntimeWarning: All-NaN slice encountered
  b = (np.nanmin(d), np.nanmax(d))
